In [2]:
import yfinance as yf
import numpy as np
import pandas as pd

https://aroussi.com/post/python-yahoo-finance

In [3]:
etf = pd.read_csv('list_etf.csv')
list_etf = etf['TICKER'].values

In [4]:
# Let us first look at one example, to see what the data may look like, and then extrapolate to 
# the other ETFs in the list

spy = yf.Ticker("SPY")

In [5]:
spy.info

{'previousClose': 428.06,
 'regularMarketOpen': 428.87,
 'twoHundredDayAverage': 398.74057,
 'trailingAnnualDividendYield': None,
 'payoutRatio': None,
 'volume24Hr': None,
 'regularMarketDayHigh': 429.74,
 'navPrice': 428.21,
 'averageDailyVolume10Day': 54630575,
 'totalAssets': 361937436672,
 'regularMarketPreviousClose': 428.06,
 'fiftyDayAverage': 420.06516,
 'trailingAnnualDividendRate': None,
 'open': 428.87,
 'toCurrency': None,
 'averageVolume10days': 54630575,
 'expireDate': None,
 'yield': 0.013300001,
 'algorithm': None,
 'dividendRate': None,
 'exDividendDate': None,
 'beta': None,
 'circulatingSupply': None,
 'startDate': None,
 'regularMarketDayLow': 428.8,
 'priceHint': 2,
 'currency': 'USD',
 'regularMarketVolume': 13805205,
 'lastMarket': None,
 'maxSupply': None,
 'openInterest': None,
 'marketCap': None,
 'volumeAllCurrencies': None,
 'strikePrice': None,
 'averageVolume': 68115415,
 'priceToSalesTrailing12Months': None,
 'dayLow': 428.8,
 'ask': 429.49,
 'ytdReturn'

In [6]:
spy.history(period='max')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1993-01-29,25.90,25.90,25.77,25.88,1003200,0.0,0
1993-02-01,25.90,26.07,25.90,26.07,480500,0.0,0
1993-02-02,26.05,26.14,25.99,26.12,201300,0.0,0
1993-02-03,26.16,26.42,26.14,26.40,529400,0.0,0
1993-02-04,26.49,26.57,26.20,26.51,531500,0.0,0
...,...,...,...,...,...,...,...
2021-06-25,425.90,427.09,425.55,426.61,58067900,0.0,0
2021-06-28,427.17,427.65,425.89,427.47,53159600,0.0,0
2021-06-29,427.88,428.56,427.13,427.70,35970500,0.0,0


In [7]:
# This may come in handy
spy.institutional_holders

,0,1
0,Net Assets,361.94B
1,NAV,428.21
2,PE Ratio (TTM),NaN
3,Yield,1.33%
4,YTD Daily Total Return,15.04%
5,Beta (5Y Monthly),1.00
6,Expense Ratio (net),0.09%
7,Inception Date,1993-01-22


In [8]:
print(spy.recommendations)

None


In [9]:
len(list_etf)

58

In [10]:
prices_dict = {} 
for i in range(len(list_etf)):
    prices_dict[list_etf[i]] = yf.Ticker(list_etf[i]).history(period="max")

- VEA : No data found, symbol may be delisted
- GSIE : No data found, symbol may be delisted
- BGRN : No data found, symbol may be delisted
- IXN : No data found, symbol may be delisted


In [11]:
prices_dict[list_etf[0]]

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2012-01-12,41.35,41.35,41.35,41.35,100,0.0,0
2012-01-13,41.35,41.35,41.35,41.35,0,0.0,0
2012-01-17,41.35,41.35,41.35,41.35,0,0.0,0
2012-01-18,41.35,41.35,41.35,41.35,0,0.0,0
2012-01-19,42.57,42.57,42.57,42.57,300,0.0,0
...,...,...,...,...,...,...,...
2021-06-25,126.68,126.81,126.53,126.75,20400,0.0,0
2021-06-28,126.75,126.77,126.33,126.68,595900,0.0,0
2021-06-29,126.93,126.95,126.58,126.78,57200,0.0,0


In [12]:
prices.shape

NameError: name 'prices' is not defined

In [ ]:
prices

In [ ]:
# We want ETFs with at least half of the price history
prices.count(axis=0)[:10]

In [ ]:
len(prices)

In [ ]:
# Truncate to the beginningof 2011
prices2 = prices[prices.index > '2011-01-11']
prices2.head()

In [ ]:
prices2.isna().sum() / len(prices2)

In [98]:
# In case we need to convert from close price to adjusted close price, but apparently all of them are adjusted 

def calculate_adjusted_prices(df, column):
    """ 
    :param df: DataFrame with raw prices along with dividend and split_ratio
        values
    :param column: String of which price column should have adjusted prices
        created for it
    :return: DataFrame with the addition of the adjusted price column
    """
    adj_column = 'adj_' + column + '_Calc'

    # Reverse the DataFrame order, sorting by date in descending order
    df.sort_index(ascending=False, inplace=True)

    # Stock Splits with value 0 means a 1:1 relationship
    df['Stock Splits'].replace(0,1,inplace=True)
    
    # Populate Values
    price_col = df[column].values
    split_col = df['Stock Splits'].values
    
    dividend_col = df['Dividends'].values
    adj_price_col = np.zeros(len(df.index))
    adj_price_col[0] = price_col[0]
          
    for i in range(1, len(price_col)):
        adj_price_col[i] = round((adj_price_col[i - 1] + adj_price_col[i - 1] *
                   (((price_col[i] * split_col[i - 1]) -
                     price_col[i - 1] -
                     dividend_col[i - 1]) / price_col[i - 1])), 4)

    df[adj_column] = adj_price_col

    # Change the DataFrame order back to dates ascending
    df.sort_index(ascending=True, inplace=True)

    return df

In [169]:
prices.to_csv('etf_prices.csv')